# Orchestrate a RAG system

In this notebook, you'll ingest and preprocess data, create embeddings, build a vector store and index, ultimately enabling you to implement a RAG system effectively.

## Before you start

Install the necessary libraries:

In [ ]:
%pip install -qU langchain-text-splitters langchain-community langchain-openai

## Initialize components

Now you need to define the authentication values that will be used when submitting embeddings and chat completion requests through the API endpoint. 

In [39]:
import os

# Define the base URL for your Azure OpenAI Service endpoint
# Replace 'Your Azure OpenAI Service Endpoint' with your actual endpoint URL obtained previously
os.environ["AZURE_OPENAI_ENDPOINT"] = 'Your Azure OpenAI Service Endpoint'

# Define the API key for your Azure OpenAI Service
# Replace 'Your Azure OpenAI Service API Key' with your actual API key obtained previously
os.environ["AZURE_OPENAI_API_KEY"] = 'Your Azure OpenAI Service API Key'

# Define the API version to use for the Azure OpenAI Service
os.environ["OPENAI_API_VERSION"] = '2024-08-01-preview'

os.environ["LANGSMITH_TRACING"] = "false" # This will deactivate the logging traces feature of LangChain as it is not required in this exercise

# Define the names of the models deployed in your Azure OpenAI Service
llm_name = 'gpt-4'
embeddings_name = 'text-embedding-ada-002'

Next, you need to initialize the components that will be used from LangChain's suite of integrations:

In [40]:
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import InMemoryVectorStore

llm = AzureChatOpenAI(azure_deployment=llm_name)
embeddings = AzureOpenAIEmbeddings(azure_deployment=embeddings_name)
vector_store = InMemoryVectorStore(embeddings)


## Create indexing pipeline

First, you need to load your dataset to begin the indexing process:

In [ ]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(file_path='./app_hotel_reviews.csv',
    csv_args={
    'delimiter': ',',
    'fieldnames': ['Hotel Name', 'User Review']
})

docs = loader.load()

print(docs[1].page_content)

Next, you need to split the documents into chunks for embedding and vector storage:

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    add_start_index=True,
)
all_splits = text_splitter.split_documents(docs)

print(f"Split documents into {len(all_splits)} sub-documents.")

Now you need to embed the contents of each text chunk and insert these embeddings into a vector store so that you can search over them to retrieve relevant documents during query.

In [ ]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

## Retrieve documents and generate answers

Now you can test the RAG application. It will take a user question, search for documents relevant to that question, pass the retrieved documents and initial question to a model, and return an answer. 

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
question = "Where can I stay in London?"

retrieved_docs = vector_store.similarity_search(question)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
prompt = prompt.invoke({"question": question, "context": docs_content})
answer = llm.invoke(prompt)

print(answer.content)

## Conclusion

In this exercise you built a typical RAG system with its main components. By using your own documents to inform a model's responses, you provide grounding data used by the LLM when it formulates a response. For an enterprise solution, that means that you can constrain generative AI to your enterprise content.

## Clean up

If you've finished the exercise, you should delete the resources you have created to avoid incurring unnecessary Azure costs.

1. Return to the browser tab containing the Azure portal (or re-open the [Azure portal](https://portal.azure.com?azure-portal=true) in a new browser tab) and view the contents of the resource group where you deployed the resources used in this exercise.
1. On the toolbar, select **Delete resource group**.
1. Enter the resource group name and confirm that you want to delete it.